In [20]:
import cv2
import numpy as np 
from collections import deque

In [7]:
buffer_size = 16 # Deque boyutu 
pts = deque(maxlen = buffer_size)
# Mavi Renk Aralığı HSV
blueLower = (84, 98, 0)
blueUpper = (179, 255, 255)

In [21]:
buffer_size = 16 # Deque boyutu 
pts = deque(maxlen = buffer_size)
# Yeşil Renk Aralığı HSV(ton,doygunluk,parlaklık)
blueLower = (57, 52, 0)
blueUpper = (125, 240, 255)

In [15]:
# HSV formatında kamera
cap = cv2.VideoCapture(0)
cap.set(3,960) # Genişlik
cap.set(4,480) # Yükseklik

while True:
    success, imgOriginal = cap.read()
    
    if success:
        
        # blur 
        blurred = cv2.GaussianBlur(imgOriginal, (11,11), 0)
        
        # HSV
        hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
        cv2.imshow("hsv image",hsv)
    if cv2.waitKey(10) & 0xFF == ord("q"): break
cap.release() 
cv2.destroyAllWindows()

In [9]:
# Maskeleme işlemi yapılmış hali
cap = cv2.VideoCapture(0)
cap.set(3,960) # Genişlik
cap.set(4,480) # Yükseklik

while True:
    success, imgOriginal = cap.read()
    
    if success:
        
        # blur (frame'in gürültüsünü azaltıyoruz.)
        blurred = cv2.GaussianBlur(imgOriginal, (11,11), 0)
        
        # HSV
        hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
        
        mask = cv2.inRange(hsv, blueLower, blueUpper)
        
        # Maskenin etrafında kalan görüntüleri sil (erozyon ve genişleme uyguladık.)
        mask = cv2.erode(mask, None, iterations = 2)
        mask = cv2.dilate(mask, None, iterations = 2)
        # cv2.imshow("mask + erozyon ve genişleme",mask)
        cv2.imshow("mask image",mask)
    if cv2.waitKey(10) & 0xFF == ord("q"): break
cap.release() #serbest bırak
cv2.destroyAllWindows()


In [22]:
# capture
cap = cv2.VideoCapture(0)
cap.set(3,960) # Genişlik
cap.set(4,480) # Yükseklik

while True:
    success, imgOriginal = cap.read()
    
    if success:
        
        # blur (frame'in gürültüsünü azaltıyoruz.)
        blurred = cv2.GaussianBlur(imgOriginal, (11,11), 0)
        
        # HSV
        hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
        
        # mavi için maske oluştur
        mask = cv2.inRange(hsv, blueLower, blueUpper)
        
        # Maskenin etrafında kalan görüntüleri sil (erozyon ve genişleme uyguladık.)
        mask = cv2.erode(mask, None, iterations = 2)
        mask = cv2.dilate(mask, None, iterations = 2)
        # cv2.imshow("mask + erozyon ve genişleme",mask)
        
        # kontur
        (contours,_) = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        center = None
        if len(contours) > 0: # eğer sıfırdan büyükse bir şeyler bulmuşuz demektir.
            
            # en büyük konturu al (en yakın nesneyi tespit eder (yani en büyük olanı) )
            c = max(contours, key = cv2.contourArea) # En büyük konturu seçtiğimiz için iki nesneden kameraya yakın olanı tespit eder.
            
            # dikdörtgene çevir
            rect = cv2.minAreaRect(c)
            
            ((x,y), (width,height), rotation) = rect # rect bir tuple'dır.
            
            s = "x: {}, y: {}, width: {}, height: {}, rotation: {}".format(np.round(x),np.round(y),np.round(width),np.round(height),np.round(rotation))
            print(s)
            
            # Kutucuk
            
            box = cv2.boxPoints(rect)
            box = np.int64(box)
            
            # Moment (görüntünün merkezini bulmamızı sağlar.)
            M = cv2.moments(c)
            center = (int(M["m10"]/M["m00"]), int(M["m01"]/M["m00"]))
            
            # Konturu çizdir : sarı
            cv2.drawContours(imgOriginal, [box], 0, (0,255,255), 2)
            
            # Merkeze bir tane nokta çiz (pembe)
            cv2.circle(imgOriginal, center, 5, (255,0,255), -1) # -1 noktanın içini doldur demek.
            
            # Bilgileri ekrana yazdır
            cv2.putText(imgOriginal, s, (25,25), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255,255,255), 2)
            
        # deque (başta büyüklüğünü 16 belirlediğimiz için 16 noktayı alır) (küçük bir nesne takibi yaptık)    
        pts.appendleft(center)
        for i in range(1, len(pts)):
            if pts[i-1] is None or pts[i] is None: continue
                
            cv2.line(imgOriginal, pts[i-1], pts[i],(0,255,0),3)    
        cv2.imshow("Orijinal Tespit",imgOriginal)      
    # cv2.imshow("mask image",mask)
    if cv2.waitKey(10) & 0xFF == ord("q"): break
cap.release() #serbest bırak
cv2.destroyAllWindows()

x: 157.0, y: 455.0, width: 68.0, height: 76.0, rotation: 74.0
x: 158.0, y: 456.0, width: 71.0, height: 76.0, rotation: 73.0
x: 158.0, y: 456.0, width: 71.0, height: 76.0, rotation: 73.0
x: 157.0, y: 456.0, width: 72.0, height: 75.0, rotation: 72.0
x: 157.0, y: 456.0, width: 72.0, height: 75.0, rotation: 72.0
x: 180.0, y: 449.0, width: 62.0, height: 58.0, rotation: 88.0
x: 160.0, y: 457.0, width: 67.0, height: 72.0, rotation: 70.0
x: 160.0, y: 457.0, width: 67.0, height: 72.0, rotation: 70.0
x: 158.0, y: 455.0, width: 66.0, height: 74.0, rotation: 76.0
x: 158.0, y: 455.0, width: 66.0, height: 74.0, rotation: 76.0
x: 627.0, y: 336.0, width: 286.0, height: 24.0, rotation: 90.0
x: 627.0, y: 336.0, width: 286.0, height: 24.0, rotation: 90.0
x: 588.0, y: 328.0, width: 303.0, height: 101.0, rotation: 90.0
x: 550.0, y: 317.0, width: 324.0, height: 179.0, rotation: 90.0
x: 550.0, y: 317.0, width: 324.0, height: 179.0, rotation: 90.0
x: 566.0, y: 296.0, width: 365.0, height: 145.0, rotation: 90.

x: 253.0, y: 205.0, width: 186.0, height: 248.0, rotation: 15.0
x: 253.0, y: 205.0, width: 186.0, height: 248.0, rotation: 15.0
x: 263.0, y: 202.0, width: 159.0, height: 239.0, rotation: 14.0
x: 263.0, y: 202.0, width: 159.0, height: 239.0, rotation: 14.0
x: 270.0, y: 214.0, width: 145.0, height: 267.0, rotation: 16.0
x: 270.0, y: 214.0, width: 145.0, height: 267.0, rotation: 16.0
x: 273.0, y: 206.0, width: 155.0, height: 251.0, rotation: 18.0
x: 273.0, y: 206.0, width: 155.0, height: 251.0, rotation: 18.0
x: 269.0, y: 208.0, width: 163.0, height: 251.0, rotation: 17.0
x: 269.0, y: 208.0, width: 163.0, height: 251.0, rotation: 17.0
x: 269.0, y: 208.0, width: 163.0, height: 251.0, rotation: 17.0
x: 251.0, y: 254.0, width: 182.0, height: 345.0, rotation: 17.0
x: 251.0, y: 254.0, width: 182.0, height: 345.0, rotation: 17.0
x: 246.0, y: 256.0, width: 194.0, height: 347.0, rotation: 17.0
x: 246.0, y: 256.0, width: 194.0, height: 347.0, rotation: 17.0
x: 257.0, y: 216.0, width: 198.0, height

x: 307.0, y: 299.0, width: 298.0, height: 361.0, rotation: 0.0
x: 307.0, y: 299.0, width: 298.0, height: 361.0, rotation: 0.0
x: 306.0, y: 300.0, width: 296.0, height: 360.0, rotation: 1.0
x: 306.0, y: 300.0, width: 296.0, height: 360.0, rotation: 1.0
x: 306.0, y: 300.0, width: 296.0, height: 360.0, rotation: 1.0
x: 305.0, y: 300.0, width: 297.0, height: 360.0, rotation: 0.0
x: 305.0, y: 300.0, width: 297.0, height: 360.0, rotation: 0.0
x: 305.0, y: 300.0, width: 297.0, height: 360.0, rotation: 0.0
x: 305.0, y: 300.0, width: 297.0, height: 360.0, rotation: 0.0
x: 306.0, y: 299.0, width: 360.0, height: 299.0, rotation: 90.0
x: 306.0, y: 299.0, width: 360.0, height: 299.0, rotation: 90.0
x: 306.0, y: 299.0, width: 360.0, height: 299.0, rotation: 90.0
x: 328.0, y: 299.0, width: 360.0, height: 345.0, rotation: 90.0
x: 328.0, y: 299.0, width: 360.0, height: 345.0, rotation: 90.0
x: 307.0, y: 300.0, width: 358.0, height: 300.0, rotation: 90.0
x: 307.0, y: 300.0, width: 358.0, height: 300.0, 

x: 235.0, y: 442.0, width: 74.0, height: 102.0, rotation: 90.0
